In [4]:
import os
from pyprojroot import here
import pandas as pd
import chromadb
from openai import AzureOpenAI
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
print(load_dotenv())

import os

# ROOT_DIR = os.path.dirname(os.path.abspath(__file__))

# SUST_DB_PATH = os.path.join(ROOT_DIR, 'Data/DB/Sustain.db')
# CHRIS_DB_PATH = os.path.join(ROOT_DIR, 'Data/DB/Chrismas.db')

True


In [9]:
from langchain_community.utilities import SQLDatabase

sust_db = SQLDatabase.from_uri(F"sqlite:///../Data/DB/Sustain.db")
chris_db = SQLDatabase.from_uri(F"sqlite:///../Data/DB/Chrismas.db")
# print(sust_db.dialect)
# print(sust_db.get_usable_table_names())

In [10]:


# Query to get the entire schema
sust_res = sust_db.run("SELECT sql FROM sqlite_master WHERE type='table';")
chris_res = chris_db.run("SELECT sql FROM sqlite_master WHERE type='table';")




In [31]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

class SubmitFinalAnswer(BaseModel):
    """Submit the final answer to the user based on the query results."""

    final_answer: str = Field(..., description="The final answer to the user")


Master_Agent = f"""
    You are a Business Analyst who recieved this question from client. You have two SQL agents one is for Chrismas.db with schema : {chris_res} , and second for Sustain.db with schema : {sust_res}. You have to decide which database above question belongs to. If it belongs to both DB redifine question in two questions where one question is explicit for one and second for another.
    Please give output in following format always:
    
    Orignal Question:

    Sustain.db Question: 

    Chrismas.db Question:

    Response:

    If question belongs to only one db keep other option empty.
    
    
"""


query_gen_prompt = ChatPromptTemplate.from_messages(
    [("system", Master_Agent), ("placeholder", "{messages}")]
)
query_gen = query_gen_prompt | ChatOpenAI(model="gpt-4o", temperature=0).bind_tools(
    [SubmitFinalAnswer]
)

res = query_gen.invoke({"messages": ["what is this application about?"]})


In [32]:
res

AIMessage(content='This application is designed to help determine which database a given question belongs to, based on the schema of two databases: Chrismas.db and Sustain.db. If a question belongs to both databases, the application will redefine the question into two separate questions, each specific to one database. The output format includes the original question, the question for Sustain.db, the question for Chrismas.db, and the response.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 2692, 'total_tokens': 2775, 'prompt_tokens_details': {'cached_tokens': 2560}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6b68a8204b', 'finish_reason': 'stop', 'logprobs': None}, id='run-a0c6eaea-a2b3-4f5c-8834-1f33b3d2c8f5-0', usage_metadata={'input_tokens': 2692, 'output_tokens': 83, 'total_tokens': 2775})

In [28]:
res = res.dict()

In [30]:
len(res['content'].split('\n'))

1

In [34]:
'tanmay jagtap' in 'mycfhsdkvnfd cfdc  cfdc tanmay jagtap'

True

In [36]:
res = """You have to decide which database above question belongs to. 
                If it belongs to both DB redifine question in two questions where one question is explicit for one and second for another.
                Please give output in following format always:
                
                Orignal Question: fdvdsv

                Sustain.db Question: vdsvsdv

                Chrismas.db Question:

                Response: vsdvsdv"""

responces = res.split('\n')

main_res = {'Orignal Question': '','Sustain.db Question':'','Chrismas.db':'','Response':''}


for responce in responces:
    if any(responce in main_res.keys ):
        res_sep = responce.split(':')
        main_res[res_sep[0]] = res_sep[1]

main_res

TypeError: 'in <string>' requires string as left operand, not builtin_function_or_method